In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
state_code = "10"
category_code = "PE"
page_num = 1


In [3]:
BASE_URL = "https://www.halal.gov.my/v4"
data_key = "ZGlyZWN0b3J5L2luZGV4X2RpcmVjdG9yeTs7Ozs"
page_data_page_num = f"/index.php?data={data_key}=&negeri={state_code}&category={category_code}&page={page_num}&cari="
url_detail_company = f"/directory/slm_viewdetail.php?comp_code=COMP-20111108-110841&type=C&ty=CO" # can get by scraping

In [4]:
BASE_URL + page_data_page_num

'https://www.halal.gov.my/v4/index.php?data=ZGlyZWN0b3J5L2luZGV4X2RpcmVjdG9yeTs7Ozs=&negeri=10&category=PE&page=1&cari='

In [7]:
resp = requests.get("https://myehalal.halal.gov.my/portal-halal/v1//index.php?data=ZGlyZWN0b3J5L2luZGV4X2RpcmVjdG9yeTs7Ozs=&negeri=01&category=PE&page=1&ty=CO")
soup = BeautifulSoup(resp.content, "html.parser")

## Find page number

In [13]:
def get_last_page(soup):
    last_page = int(soup.find("table").find_all("a")[-1].attrs["href"].split("=")[-1])
    return last_page

In [15]:
x = 3
try:
  x > 3
except:
  print("Something went wrong")
else:
  print("Nothing went wrong")
finally:
  print("The try...except block is finished")

Nothing went wrong
The try...except block is finished


In [10]:
soup.find("table").find_all("a")[-1].text

'2'

# Find the company details

## Get Company Name, Company Address, Brand, detailed_copmany_link

In [14]:
def address_to_text(list_address):
    adrress_text = "".join([str(i).replace("<br/>"," ") for i in list_address])
    return adrress_text

In [15]:
def get_company_overview_info(company):
    company_dict = {}
    name = company.find("span",{"class": "company-name"}).text.strip()
    address_text = address_to_text(company.find("span",{"class": "company-address"}).contents)
    brand_name = company.find("span",{"class":"company-brand"}).text.split("JENAMA:")[-1].strip()
    halal_status =  company.find("i")
    if halal_status != None:
        status = halal_status.text.split("HALAL :")[-1].strip()
    else:
        status = None
    detailed_company_link = company.find("img").attrs["onclick"].split('\'')[1]
    company_dict["company_name"] = name
    company_dict["company_address"] = address_text
    company_dict["company_brand"] = brand_name
    company_dict["company_halal_status"] = status
    company_dict["company_info_url"] = detailed_company_link
    return company_dict

In [16]:
def get_all_company_overview_info(li_tag):
    all_list = []
    for company in li_tag.find_all("li",{"class": "clearfix search-result-data"}):
        all_list.append(get_company_overview_info(company))
    return all_list

In [23]:
a = pd.DataFrame(get_all_company_overview_info(soup))

In [25]:
a.company_info_url.values

array(['directory/slm_viewdetail.php?comp_code=COMP-20210406-174638&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20050613-180717&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20150724-200540&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20200625-111528&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20191129-130028&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20090414-130143&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20050613-153534&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20150813-220601&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20160721-141504&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20210327-162449&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20141209-114957&type=C&ty=CO',
       'directory/slm_viewdetail.php?comp_code=COMP-20180305-161848&type=C&ty=CO',
    

### Run whole process until this function
Run for selangor and PE only

In [18]:
BASE_URL + page_data_page_num

'https://www.halal.gov.my/v4/index.php?data=ZGlyZWN0b3J5L2luZGV4X2RpcmVjdG9yeTs7Ozs=&negeri=10&category=PE&page=1&cari='

In [19]:
%%time
state_code = "10"
category_code = "PE"
page_num = 1
page_data_page_num = f"/index.php?data={data_key}=&negeri={state_code}&category={category_code}&page={page_num}&cari="
resp = requests.get(BASE_URL + page_data_page_num)
soup = BeautifulSoup(resp.content, "html.parser")
last_page = get_last_page(soup)

all_list_data = []
for page_num in range(1,last_page+1):
    print(page_num)
    page_data_page_num = f"/index.php?data={data_key}=&negeri={state_code}&category={category_code}&page={page_num}&cari="
    if page_num == 1:
        all_list_data.extend(get_all_company_overview_info(soup))
    else:
        resp = requests.get(BASE_URL + page_data_page_num)
        new_soup = BeautifulSoup(resp.content, "html.parser")
        all_list_data.extend(get_all_company_overview_info(new_soup))

KeyError: 'href'

In [20]:
pd.DataFrame(all_list_data)

NameError: name 'all_list_data' is not defined

In [331]:
page_data_page_num = f"/index.php?data={data_key}=&negeri={state_code}&category={category_code}&page={page_num}&cari="

In [26]:
detailed_company_link = "directory/slm_viewdetail.php?comp_code=COMP-20210406-174638&type=C&ty=CO"

## Detailed Company Info

In [27]:
BASE_URL+"/"+detailed_company_link

'https://www.halal.gov.my/v4/directory/slm_viewdetail.php?comp_code=COMP-20210406-174638&type=C&ty=CO'

In [28]:
resp_dc = requests.get(BASE_URL+"/"+detailed_company_link)

In [29]:
soup_dc = BeautifulSoup(resp_dc.content, "html.parser")

In [30]:
company_info = soup_dc.find_all("tr",{"valign":"top"})[1:]

In [31]:
company_info_dict = {}
for row in company_info:
    key = row.find("b").text.strip().replace(" :","")
    value = row.find_next("td").find_next("td").text.strip()
    company_info_dict[key] = value

In [32]:
company_info_dict 

{'Name': '9 KOPITIAM SDN. BHD.',
 'Address': 'No. G-10, Silk Residence,Jalan Sutera, Lebuhraya Silk43200, CherasSelangor',
 'State': 'Selangor',
 'Phone No': '0145125771',
 'Fax No.': '',
 'e-mail': '9kopitiam.halal@gmail.com',
 'Website': '',
 'Reference No.': 'JAKIM.700-2/3/2 069-06/2021',
 'Officer': 'Azlina Bt. AhmadSiti Farah Binti Ahmad'}

In [33]:
table_data = soup_dc.find_all("table",{"width":"100%"})[1]

In [35]:
data_list

[{'product': 'A01: NASI LEMAK', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A02: NASI PUTIH', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A03: CURRY CHICKEN', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A04: CURRY POTATOES', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A05: RENDANG CHICKEN', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A06: POWER 9 CHICKEN', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A07: SAMBAL SPECIAL', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A08: HOMEMADE KAYA', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'B01: HAINAN BREAD WITH HOMEMADE KAYA + BUTTER ( TOAST)',
  'brand': '',
  'expiry_date': '29/02/2024'},
 {'product': 'B02: HAINAN BREAD WITH HOMEMADE KAYA ( TOAST)',
  'brand': '',
  'expiry_date': '29/02/2024'},
 {'product': 'B03: HAINAN BREAD WITH HOMEMADE KAYA + BUTTER ( STEAM)',
  'brand': '',
  'expiry_date': '29/02/2024'},
 {'product': 'B04: HAINAN BREAD

In [34]:
data_list = []
for data_row in table_data.find_all("tr")[2:]:
    data = data_row.find_all("td")
    data_dict = {}
    data_dict["product"] = data[1].text.strip()
    data_dict["brand"] = data[2].text.strip()
    data_dict["expiry_date"] = data[3].text.strip()
    data_list.append(data_dict)
    print(data[1].text.strip())
    print(data[2].text.strip())
    print(data[3].text.strip())

A01: NASI LEMAK

29/02/2024
A02: NASI PUTIH

29/02/2024
A03: CURRY CHICKEN

29/02/2024
A04: CURRY POTATOES

29/02/2024
A05: RENDANG CHICKEN

29/02/2024
A06: POWER 9 CHICKEN

29/02/2024
A07: SAMBAL SPECIAL

29/02/2024
A08: HOMEMADE KAYA

29/02/2024
B01: HAINAN BREAD WITH HOMEMADE KAYA + BUTTER ( TOAST)

29/02/2024
B02: HAINAN BREAD WITH HOMEMADE KAYA ( TOAST)

29/02/2024
B03: HAINAN BREAD WITH HOMEMADE KAYA + BUTTER ( STEAM)

29/02/2024
B04: HAINAN BREAD WITH HOMEMADE KAYA(STEAM)

29/02/2024
B05: SOFT BOIL EGG ( 2 PCS)- KAMPUNG EGG

29/02/2024
B06: CROISSANT ORIGINAL

29/02/2024
B07: CROISSANT WITH HOMEMADE KAYA+BUTTER

29/02/2024
B08: '9 BULL' CROISSANT WITH CURRY POTATOES

29/02/2024
B09: CROISSANT MALAYSIA ( WITH SAMBAL) IKAN BILIS,KACANG,TIMUN,TELUR

29/02/2024
DC02: AIK CHEONG WHITE COFFEE

29/02/2024
DC03: AIK CHEONG COFFEE

29/02/2024
DC04: AIK CHEONG COFFEE C

29/02/2024
DC05: AIK CHEONG COFFEE O

29/02/2024
DC06: SIGNATURE BUTTER TEH

29/02/2024
DC07: TEH

29/02/2024
DC08: TEH 

In [182]:
company_info_dict["product_list"] = data_list

In [36]:
company_info_dict

{'Name': '9 KOPITIAM SDN. BHD.',
 'Address': 'No. G-10, Silk Residence,Jalan Sutera, Lebuhraya Silk43200, CherasSelangor',
 'State': 'Selangor',
 'Phone No': '0145125771',
 'Fax No.': '',
 'e-mail': '9kopitiam.halal@gmail.com',
 'Website': '',
 'Reference No.': 'JAKIM.700-2/3/2 069-06/2021',
 'Officer': 'Azlina Bt. AhmadSiti Farah Binti Ahmad'}

In [39]:
data_list

[{'product': 'A01: NASI LEMAK', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A02: NASI PUTIH', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A03: CURRY CHICKEN', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A04: CURRY POTATOES', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A05: RENDANG CHICKEN', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A06: POWER 9 CHICKEN', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A07: SAMBAL SPECIAL', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'A08: HOMEMADE KAYA', 'brand': '', 'expiry_date': '29/02/2024'},
 {'product': 'B01: HAINAN BREAD WITH HOMEMADE KAYA + BUTTER ( TOAST)',
  'brand': '',
  'expiry_date': '29/02/2024'},
 {'product': 'B02: HAINAN BREAD WITH HOMEMADE KAYA ( TOAST)',
  'brand': '',
  'expiry_date': '29/02/2024'},
 {'product': 'B03: HAINAN BREAD WITH HOMEMADE KAYA + BUTTER ( STEAM)',
  'brand': '',
  'expiry_date': '29/02/2024'},
 {'product': 'B04: HAINAN BREAD

In [41]:
product_df = pd.DataFrame(data_list)

In [42]:
product_df["company_name"] = company_info_dict["Name"]

,product,brand,expiry_date,company_name
0,A01: NASI LEMAK,,29/02/2024,9 KOPITIAM SDN. BHD.
1,A02: NASI PUTIH,,29/02/2024,9 KOPITIAM SDN. BHD.
2,A03: CURRY CHICKEN,,29/02/2024,9 KOPITIAM SDN. BHD.
3,A04: CURRY POTATOES,,29/02/2024,9 KOPITIAM SDN. BHD.
4,A05: RENDANG CHICKEN,,29/02/2024,9 KOPITIAM SDN. BHD.
...,...,...,...,...
84,S02: CUCUR SAYUR (VEGE),,29/02/2024,9 KOPITIAM SDN. BHD.
85,S03: SALAD VEGETABLE ROLL (VEGE),,29/02/2024,9 KOPITIAM SDN. BHD.
86,S04: FISHBALL SOUP (5PCS),,29/02/2024,9 KOPITIAM SDN. BHD.
87,W01: CLASSIC 9 HAINANESE CHICKEN CHOP,,29/02/2024,9 KOPITIAM SDN. BHD.


In [46]:
import re

In [54]:
re.findall(r'\d+', "images/4star.png")[0]

'4'

In [48]:
re.findall("images/4star.png")

TypeError: findall() missing 1 required positional argument: 'string'

In [55]:
STATE_LIST = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12",
    "13",
    "14",
    "15",
    "16",
]

In [60]:
for state in STATE_LIST[12:13]:
    print(state)

13
